In [1]:
from google.colab import drive 
drive.mount('/content/drive', force_remount=True) 

Mounted at /content/drive


In [ ]:
%unzip /content/drive/MyDrive/lastTime_balanced.zip 

In [ ]:
%pip install keras-tuner --upgrade 

In [4]:
BATCHSIZE = 16
DIR = 'lastTime_balanced' 
IMAGESIZE = (256, 256) 
seed = 42 

In [5]:
import tensorflow as tf

def dataset(): 
  trainDataset = tf.keras.utils.image_dataset_from_directory(
              DIR,
              validation_split=0.15,
              subset="training",
              seed=seed,
              image_size=IMAGESIZE, 
              batch_size=BATCHSIZE
              )

  validateDataset = tf.keras.utils.image_dataset_from_directory( 
              DIR,
              validation_split=0.15,
              subset="validation",
              seed=seed,
              image_size=IMAGESIZE,
              batch_size=BATCHSIZE
              )
  
  return trainDataset, validateDataset 

In [6]:
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import keras_tuner
 
def getModel(hp):
    model = tf.keras.applications.efficientnet.EfficientNetB5(
      include_top=False,
      weights='imagenet',
      input_tensor=None,
      input_shape=(256, 256, 3),
      # pooling=hp.Choice('pooling', ['avg', 'max']), 
      # classes=1, 
    ) 
    # noTrainingLayers = hp.Int("noTrainingLayers", 200, 550) 
    # for layer in model.layers[:noTrainingLayers]:
    #     layer.trainable = False
    # for layer in model.layers[noTrainingLayers:]:
    #     layer.trainable = True
    # if hp.Boolean('trainBatchNorm'):
    for layer in model.layers:
      if isinstance(layer, BatchNormalization) or isinstance(layer, Dropout):
        layer.trainable = False

    model = Sequential([model,
                        Flatten(name='top_flatten'),
                        Dense(500, activation='relu', name='dense_500'),
                        Dense(256, activation='relu', name='dense_256'),
                        Dense(1, activation=tf.nn.sigmoid, name='output_layer')  
    ]) 


    learning_rate=hp.Float('learning_rate', min_value=5e-5, max_value=1e-3, sampling="log") 
    model.compile(optimizer = Adam(learning_rate=learning_rate), loss = 'binary_crossentropy', metrics = ['accuracy', 'Recall', 'Precision']) 

    return model 

In [7]:
tuner = keras_tuner.RandomSearch( 
    getModel,
    objective='val_accuracy',
    max_trials=50,
    directory='content',
    project_name='EfficientNetB5') 

115263384/115263384 [==============================] - 1s 0us/step


In [8]:
trainDataset, validateDataset = dataset() 

tuner.search(trainDataset, epochs=10, validation_data=validateDataset, use_mutiprocessing=True) 
best_model = tuner.get_best_models()[0] 

Trial 1 Complete [01h 07m 36s]
val_accuracy: 0.8682376146316528

Best val_accuracy So Far: 0.8682376146316528
Total elapsed time: 01h 07m 36s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
max               |max               |pooling
0.00018427        |6.4822e-05        |learning_rate

Epoch 1/10
   6/1068 [..............................] - ETA: 6:41 - loss: 1.6097 - accuracy: 0.5938 - recall: 0.4878 - precision: 0.5263

1068/1068 [==============================] - 425s 380ms/step - loss: 0.4592 - accuracy: 0.7916 - recall: 0.7073 - precision: 0.7896 - val_loss: 0.3938 - val_accuracy: 0.8244 - val_recall: 0.8232 - val_precision: 0.7895
Epoch 2/10
1068/1068 [==============================] - 400s 375ms/step - loss: 0.3741 - accuracy: 0.8366 - recall: 0.7773 - precision: 0.8341 - val_loss: 0.3701 - val_accuracy: 0.8334 - val_recall: 0.8734 - val_precision: 0.7779
Epoch 3/10
1068/1068 [==============================] - 402s 376ms/step - loss: 0.2992 - accuracy: 0.8743 - recall: 0.8311 - precision: 0.8726 - val_loss: 0.3626 - val_accuracy: 0.8404 - val_recall: 0.7566 - val_precision: 0.8662
Epoch 4/10
1068/1068 [==============================] - 408s 382ms/step - loss: 0.2200 - accuracy: 0.9144 - recall: 0.8856 - precision: 0.9141 - val_loss: 0.3623 - val_accuracy: 0.8526 - val_recall: 0.8547 - val_precision: 0.8203
Epoch 5/10
1068/1068 [==============================] - 405s 379ms/step - loss: 0.1544 - ac

KeyboardInterrupt: ignored